In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    #!git clone https://github.com/mjwock/DeepFLaSH_Pytorch.git /content/drive/My\ Drive/FastAI_Microscopy_Segmentation
    %cd /content/drive/My\ Drive/FastAI_Microscopy_Segmentation
    #!git pull
except:
    pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/DeepFLaSH_Pytorch/FastAI2


In [ ]:
!pip install elasticdeform

  Created wheel for elasticdeform: filename=elasticdeform-0.4.6-cp36-cp36m-linux_x86_64.whl size=70444 sha256=b5d31878ff9544ddc50926a7fa0b5f009e6d9b88422d55448a16aff35646b38c
  Stored in directory: /root/.cache/pip/wheels/42/3a/94/a1d69f8b9da44826a171395e67f9f7a117f89af18f206481dd
Successfully built elasticdeform


In [ ]:
%tensorflow_version 1.x
%load_ext autoreload
%autoreload 2

from deepflash import utility, mask_converter, preprocessor

from skimage import io
from tqdm import tqdm
from tifffile import imsave
from pathlib import Path, PurePath

import numpy as np
import scipy.io
import matplotlib.pyplot as plt

import os, re, shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#SETTINGS

GENERAL

In [ ]:
IMG_PATH  = '/content/drive/My Drive//transfer/raw_images'
LABEL_PATH = '/content/drive/My Drive//transfer/prepared_labels'

SAVE_LABELS_TO = f'/content/drive/My Drive/FastAI_Microscopy_Segmentation/data/temp_data/transfer_labels'
SAVE_WEIGHTS_TO = f'/content/drive/My Drive/FastAI_Microscopy_Segmentation/data/temp_data/transfer_weights'

EL_SIZE = [635.9, 635.9] # size of image section in micrometer

CHANGEAXIS = False         # pixelwise mask representation instead of multimask
                           # if True shape is (W,H,C) instead of (C,W,H)

SAVE_SEPERATELY = True    # save masks and weights to seperate tiff files

SEED = 42

WEIGHT SETTINGS

In [ ]:
SIGMA_BAL = 10 
SIGMA_SEP = 6 

LAMBDA = 25 
V_BAL = 0.015 

v_bal_calc = utility.calc_vbal(MASK_PATH)

Foreground to background ratio is 0.08380466936869255


# READ FILES

LOAD IMAGES AND WEIGHTS

In [ ]:
images,labels,names = utility.load_images_and_labels(IMG_PATH, LABEL_PATH)

In [ ]:
img_list = [np.array(image.data) for image in images]
label_list = [np.array(label.data[0]) for label in labels]

ASSERT CLASS VALUES ARE WITHIN [0 , . . . , C - 1]

In [ ]:
values = np.unique(label_list)

print(f'Your masks contain {len(values)} classes with these unique values: {values}')

Your masks contain 2 classes with these unique values: [0 1]


In [ ]:
for i in values:
  assert i in range(len(values)), f'Your mask should not contain the number {i}. Set CONVERT_MASKS to \'True\' in next step.'

print('All values are correct!')

All values are correct!


In [ ]:
# Convert masks if values are not from 0 to C-1
CONVERT_MASKS = False

if CONVERT_MASKS:
  mask_converter.convertMasksToGrayscaleZeroToN(MASK_PATH, SAVE_LABELS_TO)
  masks = os.listdir(SAVE_LABELS_TO)
  label_list = [io.imread(f'{temp_path}/{mask}', as_gray=True).astype('int') for mask in masks]

CALCULATE WEIGHTS

In [ ]:
img_tensors = [{'rawdata': img, 'element_size_um': EL_SIZE} 
                for img in img_list]


pre = preprocessor.DataPreProcessor(element_size_um=None,
                                    border_weight_sigma_px=SIGMA_SEP,
                                    foreground_dist_sigma_px=SIGMA_BAL,
                                    border_weight_factor=LAMBDA,
                                    foreground_background_ratio=V_BAL
                                    )

weight_list = []
for i in tqdm(range(len(img_tensors))):
  sampleData, sampleLabels, sampleWeights, samplePdf = pre.generateSample(img_tensors[i], classlabels=label_list[i])
  weight_list.append(sampleWeights)  

  0%|          | 0/36 [00:00<?, ?it/s]/content/drive/My Drive/DeepFLaSH_Pytorch/FastAI2/deepflash/preprocessor.py:73: RuntimeWarning: invalid value encountered in true_divide
  dataScaled[..., c] = (dataScaled[..., c] - minValue) / (maxValue - minValue)
100%|██████████| 36/36 [02:19<00:00,  3.87s/it]


PROCESS LABELS

In [ ]:
# stacking labels
masks= [np.asarray([label_list[i],weight_list[i]]) for i in range(len(label_list))]
np.shape(masks)

(36, 2, 768, 1024)

In [ ]:
if CHANGEAXIS:
  for i, mask in enumerate(masks):
    masks[i] = np.moveaxis(mask, 0, -1)
np.shape(masks)

(36, 2, 768, 1024)

CREATE WORKING DIRECTORY

In [ ]:
if os.path.exists(f'{SAVE_LABELS_TO}'):
  shutil.rmtree(SAVE_LABELS_TO)
  print(f'Removed \'{SAVE_LABELS_TO}\'')

if os.path.exists(f'{SAVE_WEIGHTS_TO}'):
  shutil.rmtree(SAVE_WEIGHTS_TO)
  print(f'Removed \'{SAVE_WEIGHTS_TO}\'')

os.makedirs(f'{SAVE_LABELS_TO}')

if SAVE_SEPERATELY:
  os.makedirs(f'{SAVE_WEIGHTS_TO}')

Save masks

In [ ]:
for index, id in enumerate(names):
  if SAVE_SEPERATELY:
    imsave(f'{SAVE_LABELS_TO}/{id}.tif',np.array(masks[index][0]))
    imsave(f'{SAVE_WEIGHTS_TO}/{id}.tif',np.array(masks[index][1]))
    print(f'Saved label and corresponding weight {id}.tif')
  else:
    imsave(f'{SAVE_LABELS_TO}/{id}.tif',np.array(masks[index]))
    print(f'Saved label {id}.tif')

Saved label and corresponding weight huh7_ts1.tif
Saved label and corresponding weight huh7_ts10.tif
Saved label and corresponding weight huh7_ts11.tif
Saved label and corresponding weight huh7_ts2.tif
Saved label and corresponding weight huh7_ts3.tif
Saved label and corresponding weight huh7_ts4.tif
Saved label and corresponding weight huh7_ts5.tif
Saved label and corresponding weight huh7_ts6.tif
Saved label and corresponding weight huh7_ts7.tif
Saved label and corresponding weight huh7_ts8.tif
Saved label and corresponding weight huh7_ts9.tif
Saved label and corresponding weight tr1.tif
Saved label and corresponding weight tr10.tif
Saved label and corresponding weight tr11.tif
Saved label and corresponding weight tr12.tif
Saved label and corresponding weight tr13.tif
Saved label and corresponding weight tr14.tif
Saved label and corresponding weight tr15.tif
Saved label and corresponding weight tr16.tif
Saved label and corresponding weight tr17.tif
Saved label and corresponding weigh